In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import os, sys
import datetime as dt

In [2]:
main_data = pd.read_csv("C:\Boot Camp\Homework\project3\CSV export\Main_table.csv")
main_data.head()

,visitNumber,visitId,visitStartTime,date,fullVisitorId,channelGrouping,socialEngagementType,visits,hits,pageviews,...,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain
0,1,1478008845,1478008845,20161101,9352881518669559847,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Android,True,mobile,Europe,Western Europe,Germany,Hamburg,Hamburg,(not set)
1,1,1478002133,1478002133,20161101,4906813510055416703,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Asia,Eastern Asia,Taiwan,Taipei City,(not set),hinet.net
2,1,1477995977,1477995977,20161101,4091646801425702074,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,not available in demo dataset,not available in demo dataset,comunitel.net
3,1,1478034201,1478034201,20161101,5401793363522001290,Direct,Not Socially Engaged,1,1,1.0,...,Opera Mini,Android,True,mobile,Africa,Western Africa,Niger,not available in demo dataset,not available in demo dataset,(not set)
4,1,1478055650,1478055650,20161101,4440833520759626888,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Linux,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,utah.edu


In [3]:
clean_main_data=main_data[["visitId", "fullVisitorId", "date", "channelGrouping", "hits", "timeOnSite", "browser", "operatingSystem", "deviceCategory", "continent", "country", "region", "city", "networkDomain"]].copy()
clean_main_data.head()

,visitId,fullVisitorId,date,channelGrouping,hits,timeOnSite,browser,operatingSystem,deviceCategory,continent,country,region,city,networkDomain
0,1478008845,9352881518669559847,20161101,Organic Search,1,NaN,Chrome,Android,mobile,Europe,Germany,Hamburg,Hamburg,(not set)
1,1478002133,4906813510055416703,20161101,Organic Search,1,NaN,Chrome,Windows,desktop,Asia,Taiwan,Taipei City,(not set),hinet.net
2,1477995977,4091646801425702074,20161101,Organic Search,1,NaN,Chrome,Windows,desktop,Europe,Spain,not available in demo dataset,not available in demo dataset,comunitel.net
3,1478034201,5401793363522001290,20161101,Direct,1,NaN,Opera Mini,Android,mobile,Africa,Niger,not available in demo dataset,not available in demo dataset,(not set)
4,1478055650,4440833520759626888,20161101,Organic Search,1,NaN,Chrome,Linux,desktop,Americas,United States,not available in demo dataset,not available in demo dataset,utah.edu


In [20]:
clean_main_data.to_csv("templates/static/data/clean_main_data.csv", index=False)

In [15]:
clean_main_data["fullVisitorId"].count()

113971

In [16]:
grouped_data=main_data.groupby("visitId")
Nov_device_count=grouped_data["deviceCategory"].head().value_counts()
total_counts=grouped_data["deviceCategory"].head().count()
Nov_percentage=[]
for a in Nov_device_count:
    result=round(a/total_counts*100,2)
    Nov_percentage.append(result)
Nov_devices=pd.DataFrame({"Device_Name":Nov_device_count.keys(), "Device_Count":Nov_device_count.values, "Device_Percentage": Nov_percentage})
Nov_devices.head()

,Device_Name,Device_Count,Device_Percentage
0,desktop,95496,83.79
1,mobile,15910,13.96
2,tablet,2564,2.25


In [19]:
Nov_devices.to_csv("templates/static/data/Nov_devices.csv", index=False)
#Nov_devices.to_json("data/Nov_devices.json", orient="records")

In [21]:
black_friday_group=main_data[(main_data["date"]==20161125)].groupby("visitId")
bf_device_count=black_friday_group["deviceCategory"].head().value_counts()
bf_total_device=black_friday_group["deviceCategory"].head().count()
bf_percentage=[]
for device in bf_device_count:
    result=round(device/bf_total_device*100,2)
    bf_percentage.append(result)
bf_devices=pd.DataFrame({"Device_Name":bf_device_count.keys(), "Device_Count":bf_device_count.values, "Device_Percentage": bf_percentage}) 
bf_devices   

,Device_Name,Device_Count,Device_Percentage
0,desktop,3024,80.45
1,mobile,623,16.57
2,tablet,112,2.98


In [22]:
bf_devices.to_csv("templates/static/data/BlackFriday_devices.csv", index=False)
#bf_devices.to_json("data/BlackFriday_devices.json", orient="records")

In [36]:
main_data["date"]=pd.to_datetime(main_data["date"].astype(str), format="%Y%m%d")
main_data.head()

,visitNumber,visitId,visitStartTime,date,fullVisitorId,channelGrouping,socialEngagementType,visits,hits,pageviews,...,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain
0,1,1478008845,1478008845,2016-11-01,9352881518669559847,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Android,True,mobile,Europe,Western Europe,Germany,Hamburg,Hamburg,(not set)
1,1,1478002133,1478002133,2016-11-01,4906813510055416703,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Asia,Eastern Asia,Taiwan,Taipei City,(not set),hinet.net
2,1,1477995977,1477995977,2016-11-01,4091646801425702074,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,not available in demo dataset,not available in demo dataset,comunitel.net
3,1,1478034201,1478034201,2016-11-01,5401793363522001290,Direct,Not Socially Engaged,1,1,1.0,...,Opera Mini,Android,True,mobile,Africa,Western Africa,Niger,not available in demo dataset,not available in demo dataset,(not set)
4,1,1478055650,1478055650,2016-11-01,4440833520759626888,Organic Search,Not Socially Engaged,1,1,1.0,...,Chrome,Linux,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,utah.edu


In [68]:
unique_visits=main_data.groupby("fullVisitorId")
unique_daily_visit=unique_visits["date"].head().value_counts()
unique_daily_visit_df=pd.DataFrame({"Visit_Date":unique_daily_visit.keys(), "Visit_Count":unique_daily_visit.values}) 
unique_daily_visit_df=unique_daily_visit_df.sort_values(by="Visit_Date")
unique_daily_visit_df

,Visit_Date,Visit_Count
14,2016-11-01,3728
10,2016-11-02,3959
9,2016-11-03,3997
20,2016-11-04,3583
28,2016-11-05,3035
23,2016-11-06,3224
13,2016-11-07,3812
11,2016-11-08,3869
16,2016-11-09,3718
8,2016-11-10,4020


In [70]:
unique_daily_visit_df.to_csv("templates/static/data/Nov_Month_Visits.csv", index=False)